#### 비슷한 리뷰 제거
- 동일 사용자가 다수의 상품을 구매한 경우, 귀찮아서 비슷한 리뷰를 여러 개 작성.
- 이는 사실상, 하나의 정보만을 담고 있기에 얻을 수 있는 정보를 왜곡할 우려가 있음.
- 유사도 분석으로 비슷한 리뷰를 색출한 후 제거할 것.

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import pdist
from scipy.spatial.distance import squareform
from itertools import combinations
from collections import defaultdict
from operator import itemgetter

import re
import numpy as np

import pandas as pd

In [3]:
all_info = pd.read_csv("Aritaum_뽀오얀 미소 발효 립 & 아이 리무버.csv", engine = 'python')

In [4]:
all_info.head()

,user_id,channel,option,review,membership,recom_point
0,ccocco214,APP,����슜�웾 250ml,����옄洹뱀쑝濡� 遺��뱶�읇寃� 吏��썙以섏꽌 �뜲�씪由щ줈 �궗�슜�빐�슂 �뀕�뀕,�봽�씪�엫,5
1,kyh7kyh7,MOBILE,�씪諛섏슜�웾 120ml,�궗�슜�뻽�쓣 �븣 �늿�뿉 �옄洹뱀쟻�씠吏� �븡�븘�꽌 醫뗭븘�슂,�씪諛�,4
2,jyjoungs,MOBILE,����슜�웾 250ml,�씠嫄몃줈 �젙李⑺븳吏� 1�뀈�룄 �꽆��� 嫄� 媛숈븘�슂.\n�늿�뿉 �옄洹뱀...,�씪諛�,5
3,chunjiwoong,MOBILE,����슜�웾 250ml,�옒吏��썙吏�怨� 醫뗭븘�슂 �빆�긽 �씠寃껊쭔 �궗�뜥�슂! 媛뺤텛!!,�씪諛�,5
4,cyh8408,APP,����슜�웾 250ml,�븘由щ뵲����뿉�꽌 �옄二� 臾쇨굔�쓣 援щℓ�빐�슂 洹몃븣留덈떎 �뻾�궗�긽�...,�뒪留덊듃�겢�읇,5


In [4]:
# 1개 이상의 리뷰를 작성한 user_id = 1개 이상의 user_id가 기록된 사람.
def get_multi_user_id(df) : 
    id_list = []
    
    for user in df["user_id"].unique() : 
        num_of_id = df[df["user_id"] == user]
        if len(num_of_id) != 1 : 
            id_list.append(user)
        
    return id_list

In [5]:
def get_reviews(user_name, df) :
    new_df = df[df["user_id"] == user_name]
    new_df = new_df[["user_id", "review"]]
    return new_df

In [27]:
def build_doc_term_mat(ma_documents, use_idf = False):
    
    vectorizer = TfidfVectorizer(tokenizer=str.split, use_idf = use_idf)
    doc_term_mat = vectorizer.fit_transform(ma_documents)

    return doc_term_mat


def get_pairwise_eucl_dists(doc_term_mat, metric = "euclidean"):
    
    doc_term_mat = doc_term_mat.toarray()
    eucl_dists = pdist(doc_term_mat, metric = metric)
    eucl_dists = squareform(eucl_dists)
    
    return eucl_dists, metric


def get_eucl_dists_list(eucl_dists, doc_term_mat):
    
    num_docs, _ = doc_term_mat.shape
    vec_nums = range(0, num_docs)
    dists = [(i, j, eucl_dists[i, j]) for (i, j) in combinations(vec_nums, 2)]
      
    return list(sorted(dists, key = itemgetter(2)))



In [46]:
def get_df_with_similarity(eucl_dists_list, id_review_df) : 
    
    df = pd.DataFrame(columns = ["index1", "review1", "index2", "review2", metric + " similarity"])
    for r1, r2, n in eucl_dists_list : 

        data = {
            "index1" : list(id_review_df.index)[r1],
            "review1" : list(id_review_df["review"])[r1],
            "index2" : list(id_review_df.index)[r2],
            "review2" : list(id_review_df["review"])[r2],
            metric + " similarity" : n,
        }
        df.loc[len(df)] = data
        
    return df

In [140]:
def del_similar_reviews(cosine_df) : 
    similar = cosine_df[cosine_df.iloc[ : , 4] < 0.4]
    similar_index = similar.index1 
    
    return similar_index.unique()

In [81]:
#안 정확한 것 같아서 제외
# eucl_dists, metric = get_pairwise_eucl_dists(doc_term_mat, metric = "euclidean")
# eucl_dists_list = get_eucl_dists_list(eucl_dists, doc_term_mat)   

# euclidean = get_df_with_similarity(eucl_dists_list, id_review_df)

#### cosine similarity를 기준으로 0.6미만인 review들의 index를 원래 데이터 프레임에서 제거할 것

In [147]:
all_similar_reviews = []
id_list = get_multi_user_id(all_info)
for n in range(len(id_list)) : 
    id_review_df = get_reviews(id_list[n], all_info)
    feature_documents = list(id_review_df["review"])
    doc_term_mat = build_doc_term_mat(feature_documents, use_idf = False)
    eucl_dists, metric = get_pairwise_eucl_dists(doc_term_mat, metric = "cosine")
    eucl_dists_list = get_eucl_dists_list(eucl_dists, doc_term_mat)   
    cosine = get_df_with_similarity(eucl_dists_list, id_review_df)
    similar_reviews_index = del_similar_reviews(cosine)
    all_similar_reviews.append(similar_reviews_index)

In [152]:
all_similar_reviews = [i for i in all_similar_reviews if len(i) != 0]

In [162]:
(all_info[all_info["user_id"] == "hy1004jw"]).reset_index(drop = True)

,user_id,channel,option,review,membership
0,hy1004jw,MOBILE,코코넛수,지난번 써보공 좋아서 재구매 했습니다 지난번에는 원플러스원이었는데 이번에는 세일을 ...,일반
1,hy1004jw,MOBILE,쉐어버터,지난번 써보공 좋아서 재구매 했습니다 지난번에는 원플러스원이었는데 이번에는 세일을 ...,일반
2,hy1004jw,MOBILE,히아루론산,지난번 써보공 좋아서 재구매 했습니다 지난번에는 원플러스원이었는데 이번에는 세일을 ...,일반
3,hy1004jw,MOBILE,알로에,지난번 써보공 좋아서 재구매 했습니다 지난번에는 원플러스원이었는데 이번에는 세일을 ...,일반
4,hy1004jw,MOBILE,콜라겐,지난번 써보공 좋아서 재구매 했습니다 지난번에는 원플러스원이었는데 이번에는 세일을 ...,일반
5,hy1004jw,MOBILE,쑥,지난번 써보공 좋아서 재구매 했습니다 지난번에는 원플러스원이었는데 이번에는 세일을 ...,일반
6,hy1004jw,MOBILE,레몬,무척좋아요 항상 이거쓰는데 여름철 피부보습억 아주제격입니다세일하면 더 쟁여두고싶어요...,일반
7,hy1004jw,MOBILE,쉐어버터,무척좋아요 항상 이거쓰는데 여름철 피부보습억 아주제격입니다세일하면 더 쟁여두고싶어요...,일반
8,hy1004jw,MOBILE,콜라겐,무척좋아요 항상 이거쓰는데 여름철 피부보습억 아주제격입니다세일하면 더 쟁여두고싶어요...,일반
9,hy1004jw,MOBILE,진주,무척좋아요 항상 이거쓰는데 여름철 피부보습억 아주제격입니다세일하면 더 쟁여두고싶어요...,일반


In [163]:
all_info.loc[all_similar_reviews[1]].reset_index(drop = True)

,user_id,channel,option,review,membership
0,hy1004jw,MOBILE,코코넛수,지난번 써보공 좋아서 재구매 했습니다 지난번에는 원플러스원이었는데 이번에는 세일을 ...,일반
1,hy1004jw,MOBILE,쉐어버터,지난번 써보공 좋아서 재구매 했습니다 지난번에는 원플러스원이었는데 이번에는 세일을 ...,일반
2,hy1004jw,MOBILE,레몬,무척좋아요 항상 이거쓰는데 여름철 피부보습억 아주제격입니다세일하면 더 쟁여두고싶어요...,일반
3,hy1004jw,MOBILE,쉐어버터,무척좋아요 항상 이거쓰는데 여름철 피부보습억 아주제격입니다세일하면 더 쟁여두고싶어요...,일반
4,hy1004jw,MOBILE,진주,무척좋아요 항상 이거쓰는데 여름철 피부보습억 아주제격입니다세일하면 더 쟁여두고싶어요...,일반
5,hy1004jw,MOBILE,코코넛수,무척좋아요 항상 이거쓰는데 여름철 피부보습억 아주제격입니다세일하면 더 쟁여두고싶어요...,일반
6,hy1004jw,MOBILE,꿀,무척좋아요 항상 이거쓰는데 여름철 피부보습억 아주제격입니다세일하면 더 쟁여두고싶어요...,일반
7,hy1004jw,MOBILE,쑥,무척좋아요 항상 이거쓰는데 여름철 피부보습억 아주제격입니다세일하면 더 쟁여두고싶어요...,일반
8,hy1004jw,MOBILE,알로에,지난번 써보공 좋아서 재구매 했습니다 지난번에는 원플러스원이었는데 이번에는 세일을 ...,일반
9,hy1004jw,MOBILE,콜라겐,무척좋아요 항상 이거쓰는데 여름철 피부보습억 아주제격입니다세일하면 더 쟁여두고싶어요...,일반


#### 비슷한 내용이 제거 됨을 확인할 수 있음
#### 리뷰마다 다르겠지만 동일 계정의 유사 리뷰를 제거하려고한 바가 효과가 있음을 알 수 있음

In [188]:
del_similar_reviews = np.array([ele for arr in all_similar_reviews for ele in arr])

In [191]:
del_similar_reviews = [idx for idx in all_info.index if idx not in del_similar_reviews]

In [195]:
result = all_info.loc[del_similar_reviews].reset_index(drop = True)

In [198]:
print("총 " + str(len(all_info) - len(result)) + " 개 제거")

총 723 개 제거


In [196]:
result.to_excel("reviews_after_del_similar.xlsx", index = False)